<a href="https://colab.research.google.com/github/Susanoameno/anime2021/blob/main/2022GCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# C言語とシェル

シェルコマンドを使うとき
```
!コマンド
```

## Colabとは何か？

In [ ]:
!uname -a

Linux 8d30166f8afc 5.10.133+ #1 SMP Fri Aug 26 08:44:51 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux


In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes

# Cプログラム

In [ ]:
%%writefile hello.c
#include<stdio.h>

int main()
{
    printf("Hello, world\n");
    return 0;
}


Writing hello.c


In [ ]:
!gcc -S hello.c

In [ ]:
!ls

a.out  hello.c	hello.s  sample_data


In [ ]:
!cat hello.s

	.file	"hello.c"
	.text
	.section	.rodata
.LC0:
	.string	"Hello, world"
	.text
	.globl	main
	.type	main, @function
main:
.LFB0:
	.cfi_startproc
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	leaq	.LC0(%rip), %rdi
	call	puts@PLT
	movl	$0, %eax
	popq	%rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE0:
	.size	main, .-main
	.ident	"GCC: (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0"
	.section	.note.GNU-stack,"",@progbits


# Python

In [ ]:
def fibo(n):
    if n < 3: return 1
    return fibo(n-1)+fibo(n-2)


In [ ]:
import dis

In [ ]:
dis.dis(fibo)

  2           0 LOAD_FAST                0 (n)
              2 LOAD_CONST               1 (3)
              4 COMPARE_OP               0 (<)
              6 POP_JUMP_IF_FALSE       12
              8 LOAD_CONST               2 (1)
             10 RETURN_VALUE

  3     >>   12 LOAD_GLOBAL              0 (fibo)
             14 LOAD_FAST                0 (n)
             16 LOAD_CONST               2 (1)
             18 BINARY_SUBTRACT
             20 CALL_FUNCTION            1
             22 LOAD_GLOBAL              0 (fibo)
             24 LOAD_FAST                0 (n)
             26 LOAD_CONST               3 (2)
             28 BINARY_SUBTRACT
             30 CALL_FUNCTION            1
             32 BINARY_ADD
             34 RETURN_VALUE


In [ ]:
%%writefile fibo.py

def fibo(n):
    if n < 3: return 1
    return fibo(n-1)+fibo(n-2)

if __name__ == '__main__':
    print(fibo(36))

Overwriting fibo.py


In [ ]:
!time python3 fibo.py

14930352

real	0m3.253s
user	0m3.199s
sys	0m0.015s


In [ ]:
%%writefile fibo.c
#include<stdio.h>
int fibo(int n){
    if(n < 3) return 1;
    return fibo(n-1)+fibo(n-2);
}
int main(){
    printf("%d\n", fibo(44));
}


Overwriting fibo.c


# コンパイラ最適化(Optimizer)

In [ ]:
!gcc -O4 fibo.c

In [ ]:
!time ./a.out

701408733

real	0m1.493s
user	0m1.482s
sys	0m0.001s


In [ ]:
!gcc -S fibo.c

In [ ]:
!cat fibo.s

	.file	"fibo.c"
	.text
	.globl	fibo
	.type	fibo, @function
fibo:
.LFB0:
	.cfi_startproc
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	pushq	%rbx
	subq	$24, %rsp
	.cfi_offset 3, -24
	movl	%edi, -20(%rbp)
	cmpl	$2, -20(%rbp)
	jg	.L2
	movl	$1, %eax
	jmp	.L3
.L2:
	movl	-20(%rbp), %eax
	subl	$1, %eax
	movl	%eax, %edi
	call	fibo
	movl	%eax, %ebx
	movl	-20(%rbp), %eax
	subl	$2, %eax
	movl	%eax, %edi
	call	fibo
	addl	%ebx, %eax
.L3:
	addq	$24, %rsp
	popq	%rbx
	popq	%rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE0:
	.size	fibo, .-fibo
	.section	.rodata
.LC0:
	.string	"%d\n"
	.text
	.globl	main
	.type	main, @function
main:
.LFB1:
	.cfi_startproc
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	movl	$44, %edi
	call	fibo
	movl	%eax, %esi
	leaq	.LC0(%rip), %rdi
	movl	$0, %eax
	call	printf@PLT
	movl	$0, %eax
	popq	%rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE1:
	.size	main, .-main
	.ident	"GCC: (